# Домашнє завдання: Внесення оновлень в БД і робота з транзакціями

Це ДЗ передбачене під виконання на локальній машині. Виконання з Google Colab буде суттєво ускладнене.

## Підготовка
1. Переконайтесь, що у вас встановлены необхідні бібліотеки:
   ```bash
   pip install sqlalchemy pymysql pandas matplotlib seaborn python-dotenv
   ```

2. Створіть файл `.env` з параметрами підключення до бази даних classicmodels. Базу даних ви можете отримати через

  - docker-контейнер згідно існтрукції в [документі](https://www.notion.so/hannapylieva/Docker-1eb94835849480c9b2e7f5dc22ee4df9), також відео інструкції присутні на платформі - уроки "MySQL бази, клієнт для роботи з БД, Docker і ChatGPT для запитів" та "Як встановити Docker для роботи з базами даних без терміналу"
  - або встановивши локально цю БД - для цього перегляньте урок "Опціонально. Встановлення MySQL та  БД Сlassicmodels локально".
  
  Приклад `.env` файлу ми створювали в лекції. Ось його обовʼязкове наповнення:
    ```
    DB_HOST=your_host
    DB_PORT=3306 або 3307 - той, який Ви налаштували
    DB_USER=your_username
    DB_PASSWORD=your_password
    DB_NAME=classicmodels
    ```
  Якщо ви створили цей файл під час перегляду лекції - **новий створювати не треба**. Замініть лише назву БД, або пропишіть назву в коді створення підключення (замість отримання назви цільової БД зі змінних оточення). Але переконайтесь, що до `.env` файл лежить в тій самій папці, що і цей ноутбук.

  **УВАГА!** НЕ копіюйте скрит для **створення** `.env` файлу. В лекції він наводиться для прикладу. І давалось пояснення, що в реальних проєктах ми НІКОЛИ не пишемо доступи до бази в коді. Копіювання скрипта для створення `.env` файлу сюди в ДЗ буде вважатись грубою помилкою і ми зніматимемо бали.

3. Налаштуйте підключення через SQLAlchemy до БД за прикладом в лекції.

Рекомендую вивести (відобразити) змінну engine після створення. Вона має бути не None! Якщо None - значить у Вас не підтягнулись налаштування з .env файла.

Ви також можете налаштувати параметри підключення до БД без .env файла, просто прописавши текстом в відповідних місцях. Це - не рекомендований підхід.


## Завдання

### Завдання 1: Оновлення інформації про клієнта (2 бали)

**Створіть функцію для оновлення контактної інформації клієнта за його номером** з наступними можливостями:
- Оновлення телефону клієнта
- Оновлення email (якщо поле існує в таблиці)

Опціонально, якщо вам хочеться більше практики:
- Логування змін в окрему таблицю

Використайте підхід з параметризованими запитами через `text()` та `UPDATE` оператор. Не забудьте на початку перевірити чи існує клієнт з таким номером в базі - це хороша практика.

Отримати всі колонки, які існують в таблиці ви можете наступним запитом
```
  SELECT COLUMN_NAME, DATA_TYPE
  FROM INFORMATION_SCHEMA.COLUMNS
  WHERE TABLE_NAME = 'customers'
```

Запустіть функцію і продемонструйте її роботу, запустивши SELECT, який допоможе це зробити.



In [3]:
def update_customer_contacts(customer_id, new_phone=None, new_email=None):
    with engine.begin() as conn:

        exists_query = text("SELECT 1 FROM customers WHERE customerNumber = :id")
        if not conn.execute(exists_query, {"id": customer_id}).fetchone():
            print(f"Client with ID {customer_id} is not found!")
            return

        schema_query = text("""
            SELECT COLUMN_NAME 
            FROM INFORMATION_SCHEMA.COLUMNS 
            WHERE TABLE_NAME = 'customers'
        """)
        columns = [row[0] for row in conn.execute(schema_query).fetchall()]

        updates = []
        params = {"id": customer_id}

        if new_phone:
            updates.append("phone = :phone")
            params["phone"] = new_phone
        
        if new_email and 'email' in columns:
            updates.append("email = :email")
            params["email"] = new_email
        elif new_email:
            print(f"Field 'email' is unavalable in the table 'customers'!")

        if not updates:
            print("No data to be updated.")
            return

    
        update_sql = f"UPDATE customers SET {', '.join(updates)} WHERE customerNumber = :id"
        conn.execute(text(update_sql), params)
        print(f"Customer {customer_id} data has been updated!")


In [4]:
# Prior update (Dragon Souveniers, Ltd - 148):
print("Prior update:")
print(pd.read_sql(text("SELECT customerNumber, customerName, phone FROM customers WHERE customerNumber = 148"), engine))

# Update process:
update_customer_contacts(148, new_phone="+380-44-555-55-55", new_email="dragon.souveniers@hotmail.com")

# Following update:
print("After update:")
result_df = pd.read_sql(text("SELECT customerNumber, customerName, phone FROM customers WHERE customerNumber = 148"), engine)

print(result_df)

Prior update:
   customerNumber             customerName         phone
0             148  Dragon Souveniers, Ltd.  +65 221 7555
Field 'email' is unavalable in the table 'customers'!
Customer 148 data has been updated!
After update:
   customerNumber             customerName              phone
0             148  Dragon Souveniers, Ltd.  +380-44-555-55-55


Also as per diagram, 'e-mail' field is not available in the table 'customers'. This field is only available in the table 'employees'.

### Завдання 2: Створення нового замовлення з транзакцією (5 балів)

**Реалізуйте процес створення нового замовлення** з наступними кроками в одній транзакції:
- Створення запису в таблиці `orders`
- Додавання товарних позицій в `orderdetails`
- Перевірка наявності товарів на складі
- Зменшення кількості товарів на складі

Запустіть процес з тестовими даними і продемонструйте через SELECT, що процес успішно відпрацював і були виконані необхідні операції.




In [7]:
from datetime import datetime
def create_full_order(customer_number, items):
    with engine.begin() as conn:
        res = conn.execute(text("SELECT MAX(orderNumber) + 1 FROM orders"))
        new_order_number = res.scalar()

        conn.execute(text("""
            INSERT INTO orders (orderNumber, orderDate, requiredDate, status, customerNumber)
            VALUES (:num, :date, :req_date, 'In Process', :cust_num)"""), {
            "num": new_order_number,
            "date": datetime.now(),
            "req_date": datetime.now(),
            "cust_num": customer_number})

        for i, item in enumerate(items, 1):
            product = conn.execute(text(""" SELECT quantityInStock FROM products WHERE productCode = :code"""), {"code": item['productCode']}).fetchone()

            if not product or product[0] < item['quantity']:
                raise Exception(f"Product is not available {item['productCode']}!")

            conn.execute(text("""
                INSERT INTO orderdetails (orderNumber, productCode, quantityOrdered, priceEach, orderLineNumber)
                VALUES (:order_num, :prod_code, :qty, :price, :line)"""), {
                "order_num": new_order_number,
                "prod_code": item['productCode'],
                "qty": item['quantity'],
                "price": item['priceEach'],
                "line": i})

            conn.execute(text("""
                UPDATE products 
                SET quantityInStock = quantityInStock - :qty 
                WHERE productCode = :code"""), {"qty": item['quantity'], "code": item['productCode']})

        print(f"The order {new_order_number} has been placed!")
        return new_order_number

In [8]:
# Order data:
order_items = [
    {'productCode': 'S32_2509', 'quantity': 2, 'priceEach': 50.32},
    {'productCode': 'S50_1392', 'quantity': 3, 'priceEach': 94.92}]

# Completing transaction:
order_id = create_full_order(148, order_items)

# Results:
print("Results:")

# Order check:
order_check = pd.read_sql(text(f"SELECT * FROM orders WHERE orderNumber = {order_id}"), engine)
print("Record in the table 'orders':")
print(order_check)

# Order details check:
details_check = pd.read_sql(text(f"SELECT * FROM orderdetails WHERE orderNumber = {order_id}"), engine)
print("Record in the table 'orderdetails':")
print(details_check)

# Remaining stock check
products_check = pd.read_sql(text(f"""SELECT productCode, quantityInStock FROM products WHERE productCode IN ('S32_2509', 'S50_1392')"""), engine)

print("Current remaining stock:")
print(products_check)

The order 10426 has been placed!
Results:
Record in the table 'orders':
   orderNumber   orderDate requiredDate shippedDate      status comments  \
0        10426  2025-12-20   2025-12-20        None  In Process     None   

   customerNumber  
0             148  
Record in the table 'orderdetails':
   orderNumber productCode  quantityOrdered  priceEach  orderLineNumber
0        10426    S32_2509                2      50.32                1
1        10426    S50_1392                3      94.92                2
Current remaining stock:
  productCode  quantityInStock
0    S32_2509             2872
1    S50_1392             1013
